In [1]:
import cv2
import numpy as np
import os
import random

# === CONFIGURATION ===
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_u_shape_try1/images",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_u_shape_try1/labels"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

# === SHAPE ===
u_shape = np.array([
    [-13.0, -330.76], [-65.04, -310.79], [-110.02, -273.75], [-144.99, -227.93], [-175.58, -178.2],
    [-190.85, -123.86], [-191.42, -64.8], [-186.78, -6.54], [-176.99, 51.27], [-171.0, 109.1],
    [-163.02, 166.11], [-146.94, 219.76], [-118.11, 268.13], [-72.06, 307.65], [-16.75, 319.48],
    [41.5, 317.02], [93.43, 289.99], [132.94, 248.28], [154.98, 197.09], [166.99, 141.76],
    [177.46, 84.36], [182.12, 26.1], [191.83, -31.73], [188.0, -90.8], [180.01, -147.8],
    [160.68, -200.11], [131.54, -248.35], [91.34, -291.45], [43.56, -321.7], [-13.0, -330.76]
])

# === HELPERS ===
def get_safe_valid_points(mask, margin=5):
    h, w = mask.shape
    safe_mask = np.zeros_like(mask)
    safe_mask[margin:h-margin, margin:w-margin] = mask[margin:h-margin, margin:w-margin]
    valid = np.column_stack(np.where(safe_mask == 255))
    return valid

def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 3, x_center, y_center, width, height

def is_non_overlapping(shape, mask):
    temp_mask = np.zeros_like(mask)
    cv2.fillPoly(temp_mask, [shape.astype(np.int32)], 255)
    overlap = cv2.bitwise_and(temp_mask, mask)
    return np.sum(overlap) == 0

def update_occupancy(mask, shape):
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)

def add_shadow(image, shape, light_angle_deg=45, blur=21, strength=0.4, offset_dist=10):
    angle_rad = np.deg2rad(light_angle_deg + 180)
    offset = np.array([np.cos(angle_rad), np.sin(angle_rad)]) * offset_dist
    shifted = shape + offset
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shifted.astype(np.int32)], 255)
    blurred = cv2.GaussianBlur(mask, (0, 0), blur)
    alpha = (blurred / 255.0 * strength).astype(np.float32)
    alpha = cv2.merge([alpha]*3)
    result = np.clip(image.astype(np.float32)/255.0 * (1 - alpha), 0, 1)
    return (result * 255).astype(np.uint8)

def add_soft_glow(image, shape, blur=25, strength=0.4):
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    glow = cv2.GaussianBlur(mask, (0, 0), blur)
    alpha = (glow / 255.0 * strength).astype(np.float32)
    glow_rgb = cv2.merge([alpha]*3)
    image_float = image.astype(np.float32) / 255.0
    highlighted = np.clip(image_float + glow_rgb, 0, 1)
    return (highlighted * 255).astype(np.uint8)

def reinforce_dark_core(image, shape, alpha=1.0):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    mask_3ch = cv2.merge([mask]*3) / 255.0
    dark = np.zeros_like(image)
    image_float = image.astype(np.float32)
    result = image_float * (1 - mask_3ch * alpha) + dark * (mask_3ch * alpha)
    return np.clip(result, 0, 255).astype(np.uint8)

def draw_method2_blur(image, center, scale=0.3, light_angle=45):
    shape = u_shape * scale + center
    image = add_shadow(image, shape, light_angle_deg=light_angle)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    blurred = cv2.GaussianBlur(mask, (9, 9), 3)
    alpha = blurred.astype(float) / 255.0
    alpha = cv2.merge([alpha]*3)
    result = (alpha * 0 + (1 - alpha) * image).astype(np.uint8)
    result = add_soft_glow(result, shape, blur=25, strength=0.45)
    result = reinforce_dark_core(result, shape)
    return result, shape

def draw_method3_supersample(image, center, scale=0.3, upscale=4, light_angle=45):
    h, w = image.shape[:2]
    big = cv2.resize(image, (w * upscale, h * upscale))
    shape = u_shape * scale * upscale + np.array(center) * upscale
    cv2.fillPoly(big, [shape.astype(np.int32)], (0, 0, 0))
    shape_down = shape / upscale
    out = cv2.resize(big, (w, h), interpolation=cv2.INTER_AREA)
    out = add_shadow(out, shape_down, light_angle_deg=light_angle)
    out = add_soft_glow(out, shape_down, blur=25, strength=0.45)
    out = reinforce_dark_core(out, shape_down)
    return out, shape_down

# === MAIN LOOP ===
for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue

    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    roi_mask = np.zeros((h, w), dtype=np.uint8)
    occ_mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(roi_mask, [polygon], 255)

    valid = get_safe_valid_points(roi_mask)
    if len(valid) < 2:
        print(f"⚠️ Not enough safe ROI in {fname}")
        continue

    used_centers = []
    attempts = 0
    while len(used_centers) < 2 and attempts < 200:
        center = valid[random.randint(0, len(valid) - 1)]
        shape = u_shape * 0.3 + center
        if is_non_overlapping(shape, occ_mask):
            used_centers.append(center)
            update_occupancy(occ_mask, shape)
        attempts += 1

    if len(used_centers) < 2:
        print(f"⚠️ Could only place 1 pore in {fname}")
        continue

    labels = []
    light_angle = random.uniform(0, 360)

    image, shape1 = draw_method2_blur(image, center=np.array(used_centers[0]), light_angle=light_angle)
    xmin, ymin = shape1.min(axis=0)
    xmax, ymax = shape1.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    image, shape2 = draw_method3_supersample(image, center=np.array(used_centers[1]), light_angle=light_angle)
    xmin, ymin = shape2.min(axis=0)
    xmax, ymax = shape2.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    out_img_path = os.path.join(dirs["output_images_dir"], fname)
    out_lbl_path = os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt"))

    cv2.imwrite(out_img_path, image)
    with open(out_lbl_path, 'w') as f:
        for label in labels:
            f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")

    print(f"✅ {fname}: 2 U-shape pores (blur + supersample) with soft shadow, highlight & dark core")


✅ 0c9c31fa-8-44.1.jpg: 2 U-shape pores (blur + supersample) with soft shadow, highlight & dark core


In [3]:
import cv2
import numpy as np
import os
import random

# === CONFIGURATION ===
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_u_shape_try2/images",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_u_shape_try2/labels"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

# === FINAL DISTORTED U-SHAPE (30 POINTS) ===
u_shape = np.array([
    [-13.0, -330.76], [-65.04, -310.79], [-110.02, -273.75], [-144.99, -227.93], [-175.58, -178.2],
    [-190.85, -123.86], [-191.42, -64.8], [-186.78, -6.54], [-176.99, 51.27], [-171.0, 109.1],
    [-163.02, 166.11], [-146.94, 219.76], [-118.11, 268.13], [-72.06, 307.65], [-16.75, 319.48],
    [41.5, 317.02], [93.43, 289.99], [132.94, 248.28], [154.98, 197.09], [166.99, 141.76],
    [177.46, 84.36], [182.12, 26.1], [191.83, -31.73], [188.0, -90.8], [180.01, -147.8],
    [160.68, -200.11], [131.54, -248.35], [91.34, -291.45], [43.56, -321.7], [-13.0, -330.76]
])

# === HELPERS ===
def get_safe_valid_points(mask, margin=5):
    h, w = mask.shape
    safe_mask = np.zeros_like(mask)
    safe_mask[margin:h-margin, margin:w-margin] = mask[margin:h-margin, margin:w-margin]
    valid = np.column_stack(np.where(safe_mask == 255))
    return valid

def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 3, x_center, y_center, width, height

def is_non_overlapping(shape, mask):
    temp_mask = np.zeros_like(mask)
    cv2.fillPoly(temp_mask, [shape.astype(np.int32)], 255)
    overlap = cv2.bitwise_and(temp_mask, mask)
    return np.sum(overlap) == 0

def update_occupancy(mask, shape):
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)

def add_shadow(image, shape, light_angle_deg=45, blur=21, strength=0.4, offset_dist=10):
    angle_rad = np.deg2rad(light_angle_deg + 180)
    offset = np.array([np.cos(angle_rad), np.sin(angle_rad)]) * offset_dist
    shifted = shape + offset
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shifted.astype(np.int32)], 255)
    blurred = cv2.GaussianBlur(mask, (0, 0), blur)
    alpha = (blurred / 255.0 * strength).astype(np.float32)
    alpha = cv2.merge([alpha]*3)
    result = np.clip(image.astype(np.float32)/255.0 * (1 - alpha), 0, 1)
    return (result * 255).astype(np.uint8)

def add_soft_glow(image, shape, blur=25, strength=0.4):
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    glow = cv2.GaussianBlur(mask, (0, 0), blur)
    alpha = (glow / 255.0 * strength).astype(np.float32)
    glow_rgb = cv2.merge([alpha]*3)
    image_float = image.astype(np.float32) / 255.0
    highlighted = np.clip(image_float + glow_rgb, 0, 1)
    return (highlighted * 255).astype(np.uint8)

def draw_method2_blur(image, center, scale=0.3, light_angle=45):
    shape = u_shape * scale + center
    image = add_shadow(image, shape, light_angle_deg=light_angle)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    blurred = cv2.GaussianBlur(mask, (9, 9), 3)
    alpha = blurred.astype(float) / 255.0
    alpha = cv2.merge([alpha]*3)
    result = (alpha * 0 + (1 - alpha) * image).astype(np.uint8)
    result = add_soft_glow(result, shape, blur=25, strength=0.45)
    return result, shape

def draw_method3_supersample(image, center, scale=0.3, upscale=4, light_angle=45):
    h, w = image.shape[:2]
    big = cv2.resize(image, (w * upscale, h * upscale))
    shape = u_shape * scale * upscale + np.array(center) * upscale
    cv2.fillPoly(big, [shape.astype(np.int32)], (0, 0, 0))
    shape_down = shape / upscale
    out = cv2.resize(big, (w, h), interpolation=cv2.INTER_AREA)
    out = add_shadow(out, shape_down, light_angle_deg=light_angle)
    out = add_soft_glow(out, shape_down, blur=25, strength=0.45)
    return out, shape_down

# === MAIN LOOP ===
for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue

    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    roi_mask = np.zeros((h, w), dtype=np.uint8)
    occ_mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(roi_mask, [polygon], 255)

    valid = get_safe_valid_points(roi_mask)
    if len(valid) < 2:
        print(f"⚠️ Not enough safe ROI in {fname}")
        continue

    used_centers = []
    attempts = 0
    while len(used_centers) < 2 and attempts < 200:
        center = valid[random.randint(0, len(valid) - 1)]
        shape = u_shape * 0.3 + center
        if is_non_overlapping(shape, occ_mask):
            used_centers.append(center)
            update_occupancy(occ_mask, shape)
        attempts += 1

    if len(used_centers) < 2:
        print(f"⚠️ Could only place 1 pore in {fname}")
        continue

    labels = []
    light_angle = random.uniform(0, 360)

    image, shape1 = draw_method2_blur(image, center=np.array(used_centers[0]), light_angle=light_angle)
    xmin, ymin = shape1.min(axis=0)
    xmax, ymax = shape1.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    image, shape2 = draw_method3_supersample(image, center=np.array(used_centers[1]), light_angle=light_angle)
    xmin, ymin = shape2.min(axis=0)
    xmax, ymax = shape2.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    out_img_path = os.path.join(dirs["output_images_dir"], fname)
    out_lbl_path = os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt"))

    cv2.imwrite(out_img_path, image)
    with open(out_lbl_path, 'w') as f:
        for label in labels:
            f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")

    print(f"✅ {fname}: U-shape pores added (Method 2 + 3, with glow + shadow + overlap protection)")


✅ 0c9c31fa-8-44.1.jpg: U-shape pores added (Method 2 + 3, with glow + shadow + overlap protection)


In [7]:
import cv2
import numpy as np
import os
import random

# === PATH SETUP ===
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_hooked_crescent_try3/images",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/final_hooked_crescent_try3/labels"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

# === SHAPE ===
hooked_crescent_shape = np.array([
    [-45.61, -307.3], [-99.54, -287.42], [-141.56, -244.39], [-167.56, -192.99], [-183.6, -137.46],
    [-188.6, -78.2], [-179.91, -19.63], [-162.8, 35.45], [-145.63, 90.51], [-132.2, 147.12],
    [-121.61, 204.9], [-111.57, 262.91], [-99.67, 320.16], [-68.67, 309.99], [-46.22, 257.13],
    [-7.6, 212.74], [47.17, 194.88], [105.49, 185.59], [157.36, 160.72], [196.2, 115.81],
    [219.67, 63.36], [231.84, 6.23], [233.29, -54.43], [221.32, -111.64], [198.43, -164.32],
    [163.7, -212.11], [118.12, -252.32], [68.36, -282.27], [14.46, -302.24], [-45.61, -307.3]
])

# === HELPERS ===
def get_safe_valid_points(mask, margin=5):
    h, w = mask.shape
    safe_mask = np.zeros_like(mask)
    safe_mask[margin:h-margin, margin:w-margin] = mask[margin:h-margin, margin:w-margin]
    valid = np.column_stack(np.where(safe_mask == 255))
    return valid

def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 3, x_center, y_center, width, height

def draw_method2_blur(image, center, scale=0.3):
    shape = hooked_crescent_shape * scale + center
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)
    blurred = cv2.GaussianBlur(mask, (9, 9), 3)
    alpha = blurred.astype(float) / 255.0
    alpha = cv2.merge([alpha]*3)
    result = (alpha * 0 + (1 - alpha) * image).astype(np.uint8)
    return result, shape

def draw_method3_supersample(image, center, scale=0.3, upscale=4):
    h, w = image.shape[:2]
    big = cv2.resize(image, (w * upscale, h * upscale))
    shape = hooked_crescent_shape * scale * upscale + np.array(center) * upscale
    cv2.fillPoly(big, [shape.astype(np.int32)], (0, 0, 0))
    shape_down = shape / upscale
    return cv2.resize(big, (w, h), interpolation=cv2.INTER_AREA), shape_down

def add_soft_shadow(image, shape, shadow_offset=(5, 5), shadow_blur=15, shadow_strength=0.3):
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    shifted_shape = shape + np.array(shadow_offset)
    cv2.fillPoly(mask, [shifted_shape.astype(np.int32)], 255)
    shadow = cv2.GaussianBlur(mask, (shadow_blur, shadow_blur), 0)
    shadow = (shadow_strength * shadow / 255.0).astype(np.float32)
    shadow = cv2.merge([shadow]*3)
    image_float = image.astype(np.float32) / 255.0
    shadowed = np.clip(image_float * (1 - shadow), 0, 1)
    return (shadowed * 255).astype(np.uint8)

def add_realistic_highlight(image, shape, light_dir=(-1, -1), 
                            highlight_intensity=0.6, blur_sigma=3, 
                            highlight_arc_fraction=0.25, falloff_type='gaussian'):
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [shape.astype(np.int32)], 255)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return image
    contour = contours[0]

    light_dir = np.array(light_dir, dtype=np.float32)
    light_dir /= np.linalg.norm(light_dir)

    arc_angles = []
    for i in range(len(contour)):
        p1 = contour[i][0]
        p2 = contour[(i + 1) % len(contour)][0]
        edge = p2 - p1
        normal = np.array([-edge[1], edge[0]], dtype=np.float32)
        normal /= (np.linalg.norm(normal) + 1e-6)
        dot = np.dot(normal, light_dir)
        angle = np.arccos(np.clip(dot, -1, 1))
        arc_angles.append((angle, tuple(p1), tuple(p2)))

    arc_angles.sort(key=lambda x: x[0])
    keep_count = int(len(arc_angles) * highlight_arc_fraction)
    selected_edges = arc_angles[:keep_count]

    highlight_mask = np.zeros((h, w), dtype=np.uint8)
    for _, p1, p2 in selected_edges:
        cv2.line(highlight_mask, p1, p2, 255, 1)

    if falloff_type == 'gaussian':
        highlight = cv2.GaussianBlur(highlight_mask, (0, 0), blur_sigma)
    else:
        highlight = cv2.GaussianBlur(highlight_mask, (0, 0), blur_sigma)

    highlight = (highlight / 255.0) * highlight_intensity
    highlight = cv2.merge([highlight]*3)

    img_float = image.astype(np.float32) / 255.0
    blended = np.clip(img_float + highlight, 0, 1)
    return (blended * 255).astype(np.uint8)

# === MAIN LOOP ===
for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue

    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(mask, [polygon], 255)

    valid = get_safe_valid_points(mask)
    if len(valid) < 2:
        print(f"⚠️ Not enough safe ROI in {fname}")
        continue

    used_centers = random.sample(valid.tolist(), 2)
    labels = []

    # Method 2
    image, shape1 = draw_method2_blur(image, center=np.array(used_centers[0]), scale=0.3)
    image = add_soft_shadow(image, shape1)
    image = add_realistic_highlight(image, shape1, light_dir=(-1, -1), blur_sigma=4, highlight_arc_fraction=0.25)
    xmin, ymin = shape1.min(axis=0)
    xmax, ymax = shape1.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    # Method 3
    image, shape2 = draw_method3_supersample(image, center=np.array(used_centers[1]), scale=0.3)
    image = add_soft_shadow(image, shape2)
    image = add_realistic_highlight(image, shape2, light_dir=(-1, -1), blur_sigma=4, highlight_arc_fraction=0.25)
    xmin, ymin = shape2.min(axis=0)
    xmax, ymax = shape2.max(axis=0)
    labels.append(to_yolo_bbox(xmin, ymin, xmax, ymax, w, h))

    out_img_path = os.path.join(dirs["output_images_dir"], fname)
    out_lbl_path = os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt"))

    cv2.imwrite(out_img_path, image)
    with open(out_lbl_path, 'w') as f:
        for label in labels:
            f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")

    print(f"✅ {fname}: 2 defects added (blur + supersample) with shadow + directional highlight")


✅ 0c9c31fa-8-44.1.jpg: 2 defects added (blur + supersample) with shadow + directional highlight
